In [46]:
import forallpeople as si
import numpy as np
from IPython.display import display, Markdown, Latex

si.environment("default")

eta = 0.9
n = 4
alpha_B = 0.7
p_backup = 25 * si.W
t_BACKUP = 1.8 * si.s
v_cell_max = 2.5 * si.V
r_snsc = 0.006 * si.Ohm

display(Markdown(f"&eta; = {eta}"))
display(Markdown(f"n = {n}"))
display(Latex(rf"$\alpha_B = {alpha_B}$"))
display(Markdown(f"P<sub>BACKUP</sub> = {p_backup}"))
display(Markdown(f"t<sub>BACKUP</sub> = {t_BACKUP}"))
display(Markdown(f"V<sub>CELL(MAX)</sub> = {v_cell_max}"))
display(Markdown(f"R<sub>SNSC</sub> = {r_snsc}"))

c_eol = (
    (4 * p_backup * t_BACKUP)
    / (n * eta * np.power(v_cell_max, 2))
    * np.power(
        alpha_B
        + np.sqrt(alpha_B)
        - (1 - alpha_B)
        * np.log((1 + np.sqrt(alpha_B) / np.sqrt(1 - np.sqrt(alpha_B)))),
        -1,
    )
)
display(Markdown(f"C<sub>EOL</sub> = {c_eol}"))

esr_eol = eta * (1 - alpha_B) * n * np.power(v_cell_max, 2) / (4 * p_backup)
display(Markdown(f"ESR<sub>EOL</sub> = {esr_eol}"))

v_stk_min__at_max_power = np.sqrt(4 * esr_eol * n * p_backup / eta)
display(
    Markdown(f"V<sub>STK(MIN)@max_power</sub> = {v_stk_min__at_max_power}")
)
i_peak = 0.058 * si.V / r_snsc
display(Markdown(f"I<sub>PEAK</sub> = {i_peak}"))

v_stk_min__at_current_limit = (
    p_backup / (eta * i_peak)
) + n * esr_eol * i_peak
display(
    Markdown(
        f"V<sub>STK(MIN)@current_limit</sub> = {v_stk_min__at_current_limit}"
    )
)
v_stk_min = max(v_stk_min__at_max_power, v_stk_min__at_current_limit)
display(Markdown(f"V<sub>STK(MIN)</sub> = {v_stk_min}"))


&eta; = 0.9

n = 4

<IPython.core.display.Latex object>

P<sub>BACKUP</sub> = 25.000 W

t<sub>BACKUP</sub> = 1.800 s

...

ESR<sub>EOL</sub> = 67.500 mΩ

V<sub>STK(MIN)@max_power</sub> = 5.477 V

I<sub>PEAK</sub> = 9.667 A

V<sub>STK(MIN)@current_limit</sub> = 5.484 V

V<sub>STK(MIN)</sub> = 5.484 V